# Examples of DialogEvent class and its component classes.

In [1]:
#add ../src to the path ready for python to find it
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

## Intantiate using contructors

This code shows how all of the types `Span`, `Token`, `Feature` and `DialogEvent` can all be instantiated directly via their constructors

In [2]:
from openfloor import DialogEvent, Span, Feature, Token
from datetime import datetime

span=Span(startTime=datetime.now())
text_feature=Feature(mimeType="text/plain", tokens=[Token(value="Hello, world!")])
utterance=DialogEvent(
    id="1",
    span=span,
    speakerUri="tag:userproxy.com,2025:abc123",
    features={"text":text_feature}
)
print(utterance.to_json(indent=4))

{
    "id": "1",
    "speakerUri": "tag:userproxy.com,2025:abc123",
    "span": {
        "startTime": "2025-05-09T13:40:32.502103"
    },
    "features": {
        "text": {
            "mimeType": "text/plain",
            "tokens": [
                {
                    "value": "Hello, world!"
                }
            ]
        }
    }
}


The same thing can be achieved by the following code.  The DialogEvent uses a default Span object which defaults the startTime to datetime.now().  The TextFeature class sets the mimeType to 'plainText' and also takes a parameter 'values' which is a convenient way to specify the text in an array of tokens.

In [3]:
from openfloor import TextFeature, DialogEvent

utterance=DialogEvent(
    speakerUri="tag:userproxy.com,2025:abc123",
    features={"text":TextFeature(values=["Hello, world!"])}
)
print(utterance.to_json(indent=4))

{
    "id": "de:146a4328-7a04-45d3-97c2-b0ad9f06fc53",
    "speakerUri": "tag:userproxy.com,2025:abc123",
    "span": {
        "startTime": "2025-05-09T13:40:32.509650"
    },
    "features": {
        "text": {
            "mimeType": "text/plain",
            "tokens": [
                {
                    "value": "Hello, world!"
                }
            ]
        }
    }
}


## Instantiate from dictionaries. 

This code shows how all of the types `Span`, `Token`, `Feature` and `DialogEvent` can all be instantiated from dictionaries (i.e. json importable/exportable objects)

In [4]:
from openfloor import Span, Token, Feature, DialogEvent

# Create a Span from JSON
span_json = '{"startTime": "2024-03-14T12:00:00.000000"}'
span = Span.from_json(span_json)
print(span.to_json())

# Create a Token from JSON
token_json = '{"value": "hello", "confidence": 0.95}'
token = Token.from_json(token_json)
print(token.to_json())

# Create a Feature from JSON
feature_json = '{"mimeType": "text/plain", "tokens": [{"value": "hello"}]}'
feature = Feature.from_json(feature_json)
print(feature.to_json())

# Create a DialogEvent from JSON
dialog_event_json = '{"speakerUri": "tag:userproxy.com,2025:abc123", "span": {"startTime": "2024-03-14T12:00:00.000000"}, "features": {"text": {"mimeType": "text/plain", "tokens": [{"value": "hello"}]}}}'
dialog_event = DialogEvent.from_json(dialog_event_json)
print(dialog_event.to_json())


{"startTime": "2025-05-09T13:40:32.523129"}
{"value": "hello", "confidence": 0.95}
{"mimeType": "text/plain", "tokens": [{"value": "hello"}]}
{"id": "de:92d0a62a-6916-4128-9997-905cb3368ed5", "speakerUri": "tag:userproxy.com,2025:abc123", "span": {"startTime": "2025-05-09T13:40:32.524155"}, "features": {"text": {"mimeType": "text/plain", "tokens": [{"value": "hello"}]}}}


## Intantiate from JSON file

All the objects can be instantiated from JSON files.  Here's an example of reading a whole `DialogEvent` event from file.  The types `Span`, `Token` and `Feature` can also be read from file in a similar way.

In [5]:
from openfloor import Span, Token, Feature, DialogEvent

# Read a dialog event from a file
dialog_event = DialogEvent.from_file("../sample_json/dialog_event1.json")

#print indent in pretty json format
print(dialog_event.to_json(indent=4))

{
    "id": "user-utterance-45",
    "speakerUri": "tag:userproxy.com,2025:abc123",
    "span": {
        "startTime": "2025-05-09T13:40:32.539887",
        "endOffset": "PT0S"
    },
    "features": {
        "user-request-audio": {
            "mimeType": "audio/wav",
            "tokens": [
                {
                    "valueUrl": "http://localhost:8080/ab78h50ef.wav"
                }
            ]
        },
        "user-request-text": {
            "mimeType": "text/plain",
            "tokens": [
                {
                    "value": "what is the weather forecast for tomorrow?",
                    "span": {
                        "startOffset": "PT0S",
                        "endOffset": "PT0S"
                    },
                    "confidence": 0.99,
                    "links": [
                        "$.user-request-audio.tokens[0].valueUrl"
                    ]
                },
                {
                    "value": "what is the weathe

## Resolving links between tokens

Dialog event tokens can refer to other tokens in the same dialog event.   The following code reads the same JSON file as above and finds links between tokens by resolving the Json Path links.

In [6]:
from openfloor import Span, Token, Feature, DialogEvent

# Read a dialog event from a file
dialog_event = DialogEvent.from_file("../sample_json/dialog_event1.json")

#now resolve the links
for feature_name, feature in dialog_event.features.items():
    print(f"feature: {feature}")
    tokens=feature.tokens
    for token in tokens:
        linked_values=token.linked_values(dialog_event)
        print(f"  token: {token}")
        print(f'  linked_values: {linked_values}')

feature: Feature(mimeType='audio/wav', tokens=[Token(value=None, valueUrl='http://localhost:8080/ab78h50ef.wav', span=None, confidence=None, links=[])], alternates=[], lang=None, encoding=None, tokenSchema=None)
  token: Token(value=None, valueUrl='http://localhost:8080/ab78h50ef.wav', span=None, confidence=None, links=[])
  linked_values: []
feature: Feature(mimeType='text/plain', tokens=[Token(value='what is the weather forecast for tomorrow?', valueUrl=None, span=Span(startTime=None, startOffset=datetime.timedelta(0), endTime=None, endOffset=datetime.timedelta(0)), confidence=0.99, links=['$.user-request-audio.tokens[0].valueUrl']), Token(value='what is the weather forecast for tuesday?', valueUrl=None, span=Span(startTime=None, startOffset=datetime.timedelta(0), endTime=None, endOffset=datetime.timedelta(0)), confidence=0.87, links=[])], alternates=[], lang='en', encoding='utf-8', tokenSchema=None)
  token: Token(value='what is the weather forecast for tomorrow?', valueUrl=None, sp